In [2]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

sessions= pd.read_csv('input/sessions.csv')
train_users = pd.read_csv('input/train_users_2.csv')


In [6]:
test=pd.read_csv('input/test_users.csv')

In [8]:
sessions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10567737 entries, 0 to 10567736
Data columns (total 6 columns):
user_id          object
action           object
action_type      object
action_detail    object
device_type      object
secs_elapsed     float64
dtypes: float64(1), object(5)
memory usage: 483.8+ MB


In [3]:
tmp=train_users[['id','country_destination']]
tmp.rename(columns={'id': 'user_id'}, inplace=True) 
tmp.head()

,user_id,country_destination
0,gxn3p5htnn,NDF
1,820tgsjxq7,NDF
2,4ft3gnwmtx,US
3,bjjt8pjhuk,other
4,87mebub9p4,US


In [4]:
print(tmp.groupby(['country_destination'])['user_id'].nunique().reset_index())

   country_destination  user_id
0                   AU      539
1                   CA     1428
2                   DE     1061
3                   ES     2249
4                   FR     5023
5                   GB     2324
6                   IT     2835
7                  NDF   124543
8                   NL      762
9                   PT      217
10                  US    62376
11               other    10094


In [4]:
tmp['count']=tmp['country_destination']

In [5]:
tmp.loc[tmp['count']!= 'NDF','count']=1
tmp.loc[tmp['count']== 'NDF','count']=0

tmp.head()

,user_id,country_destination,count
0,gxn3p5htnn,NDF,0
1,820tgsjxq7,NDF,0
2,4ft3gnwmtx,US,1
3,bjjt8pjhuk,other,1
4,87mebub9p4,US,1


In [6]:
sessions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10567737 entries, 0 to 10567736
Data columns (total 6 columns):
user_id          object
action           object
action_type      object
action_detail    object
device_type      object
secs_elapsed     float64
dtypes: float64(1), object(5)
memory usage: 483.8+ MB


In [7]:
sessions = pd.merge(tmp,sessions,on='user_id',how='inner')

In [8]:
print(sessions.apply(lambda x: x.nunique(),axis=0))

user_id                 73815
country_destination        12
count                       2
action                    331
action_type                 9
action_detail             128
device_type                14
secs_elapsed           256593
dtype: int64


## 1

In [9]:
grpby = sessions.groupby(['user_id'])['secs_elapsed'].sum().reset_index()
grpby.columns = ['user_id','secs_elapsed']
grpby = pd.merge(grpby,tmp,on='user_id',how='inner')
grpby.head()

,user_id,secs_elapsed,country_destination,count
0,00023iyk9l,867896.0,US,1
1,001wyh0pz8,282965.0,NDF,0
2,0028jgx1x1,297010.0,NDF,0
3,002qnbzfs5,6487080.0,US,1
4,0035hobuyj,5724670.0,US,1


In [10]:
grpby['timeclass'] = pd.qcut(grpby['secs_elapsed'], 10) #画下！
grpby[['timeclass', 'count']].groupby(['timeclass'])['count'].sum().reset_index()

,timeclass,count
0,"(-0.001, 46498.0]",1843
1,"(46498.0, 191518.0]",1850
2,"(191518.0, 374824.0]",2279
3,"(374824.0, 608254.0]",2704
4,"(608254.0, 900033.0]",2886
5,"(900033.0, 1277171.0]",3066
6,"(1277171.0, 1766269.0]",3042
7,"(1766269.0, 2474073.0]",3318
8,"(2474073.0, 3806479.0]",3473
9,"(3806479.0, 38221363.0]",3948


In [11]:
print(sessions.action_type.value_counts()) #这里对几个type重复计算订票比例！

view                1669723
data                1188495
click               1064436
-unknown-            564803
submit               357371
message_post          55567
booking_request       10695
partner_callback       7005
booking_response          2
Name: action_type, dtype: int64


In [12]:
print(sessions.action_detail.value_counts())

view_search_results              912787
p3                               610108
-unknown-                        564803
wishlist_content_update          449753
change_trip_characteristics      315766
similar_listings                 307947
user_profile                     268271
update_listing                   155351
user_social_connections          143178
header_userpic                    84877
listing_reviews                   83230
message_thread                    77839
user_wishlists                    76945
dashboard                         74060
contact_host                      56968
edit_profile                      55768
message_post                      55567
change_contact_host_dates         36799
confirm_email_link                34449
unavailable_dates                 33619
create_user                       33013
p5                                27861
user_profile_content_update       26786
login                             23858
user_listings                     21631


In [15]:
len(sessions[sessions.action_type=='booking_request'])

10695

In [14]:
sessions[sessions.action_type=='booking_request']['count'].sum()/le

9081

In [18]:
print 1.0*9081/10695*100,"% book"

84.9088359046 % book


## 2

In [46]:
sessions[sessions.action=='booking']#不知道咋回事booking了 ndf

,user_id,country_destination,action,action_type,action_detail,device_type,secs_elapsed
284210,6udv3scuxe,NDF,booking,booking_response,booking,Windows Desktop,71212.0
3645762,yxf0sm9sbw,US,booking,booking_response,booking,Windows Desktop,36905.0


In [10]:
print(sessions.groupby(['action_type'])['user_id'].nunique().reset_index())

        action_type  user_id
0         -unknown-   108415
1   booking_request    17495
2  booking_response        4
3             click   112420
4              data   125510
5      message_post    43834
6            modify      450
7  partner_callback    14428
8            submit   125538
9              view   123270


## 3

In [29]:
action_type = pd.pivot_table(sessions, index = ['user_id'],columns = ['action_type'],
                             values = 'action',aggfunc=len,fill_value=0).reset_index()

In [32]:
action_type= pd.merge(action_type,tmp,on='user_id',how='inner')

In [33]:
action_type.head()#可视化！

,user_id,-unknown-,booking_request,booking_response,click,data,message_post,partner_callback,submit,view,country_destination,count
0,00023iyk9l,0,1,0,4,9,1,1,0,21,US,1
1,001wyh0pz8,6,0,0,66,2,0,0,3,8,NDF,0
2,0028jgx1x1,1,0,0,9,5,0,0,1,15,NDF,0
3,002qnbzfs5,184,1,0,140,140,16,0,15,216,US,1
4,0035hobuyj,9,0,0,206,41,3,0,4,55,US,1


In [34]:
sessions.groupby(['device_type'])['user_id'].nunique().reset_index()

,device_type,user_id
0,-unknown-,13126
1,Android App Unknown Phone/Tablet,2757
2,Android Phone,3474
3,Blackberry,59
4,Chromebook,260
5,Linux Desktop,354
6,Mac Desktop,28893
7,Opera Phone,1
8,Tablet,848
9,Windows Desktop,24768


In [35]:
sessions.groupby(['user_id'])['device_type'].nunique().reset_index().head()#有人不止一个device

,user_id,device_type
0,00023iyk9l,2
1,001wyh0pz8,1
2,0028jgx1x1,2
3,002qnbzfs5,2
4,0035hobuyj,1


In [36]:
device_type = pd.pivot_table(sessions, index = ['user_id'],columns = ['device_type'],values = 'action',
                             aggfunc=len,fill_value=0).reset_index()

In [37]:
device_type= pd.merge(device_type,tmp,on='user_id',how='inner')
device_type.head()#可视化！

,user_id,-unknown-,Android App Unknown Phone/Tablet,Android Phone,Blackberry,Chromebook,Linux Desktop,Mac Desktop,Opera Phone,Tablet,Windows Desktop,Windows Phone,iPad Tablet,iPhone,iPodtouch,country_destination,count
0,00023iyk9l,0,0,0,0,0,0,36,0,0,0,0,0,4,0,US,1
1,001wyh0pz8,0,90,0,0,0,0,0,0,0,0,0,0,0,0,NDF,0
2,0028jgx1x1,30,0,1,0,0,0,0,0,0,0,0,0,0,0,NDF,0
3,002qnbzfs5,14,0,0,0,0,0,0,0,0,0,0,0,775,0,US,1
4,0035hobuyj,0,0,0,0,0,0,489,0,0,0,0,0,0,0,US,1


In [38]:
device_type.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73815 entries, 0 to 73814
Data columns (total 17 columns):
user_id                             73815 non-null object
-unknown-                           73815 non-null int64
Android App Unknown Phone/Tablet    73815 non-null int64
Android Phone                       73815 non-null int64
Blackberry                          73815 non-null int64
Chromebook                          73815 non-null int64
Linux Desktop                       73815 non-null int64
Mac Desktop                         73815 non-null int64
Opera Phone                         73815 non-null int64
Tablet                              73815 non-null int64
Windows Desktop                     73815 non-null int64
Windows Phone                       73815 non-null int64
iPad Tablet                         73815 non-null int64
iPhone                              73815 non-null int64
iPodtouch                           73815 non-null int64
country_destination                

In [40]:
sessions_data = pd.merge(action_type,device_type,on='user_id',how='inner')

In [41]:
sessions_data = pd.merge(sessions_data,grpby,on='user_id',how='inner')

In [42]:
sessions_data.head().T#这个想想怎么用

,0,1,2,3,4
user_id,00023iyk9l,001wyh0pz8,0028jgx1x1,002qnbzfs5,0035hobuyj
-unknown-_x,0,6,1,184,9
booking_request,1,0,0,1,0
booking_response,0,0,0,0,0
click,4,66,9,140,206
data,9,2,5,140,41
message_post,1,0,0,16,3
partner_callback,1,0,0,0,0
submit,0,3,1,15,4
view,21,8,15,216,55
